1. Implement two classification algorithms for detecting faces and classifying digits:
(a) Perceptron
(b) Two-layer Neural Network (input layer, one hidden layer, output)
2. Design the features for each of the two problems, and write a program for extracting the features from each image.
You can also use the raw pixels directly as features (and avoid the process of designing features altogether) if that
results in a better classification accuracy.
3. Train the two algorithms on the part of the data set that is reserved for training. First, use only 10% of the data points
that are reserved for training, then 20%, 30%, 40%, 50%, 60%, 70%, 80%, 90%, and finally 100%. All the results
should a function of the number of data points used for training.
4. Compare the performances of the two algorithms using the part of the data set that is reserved for testing, and report:
• The time needed for training as a function of the number of data points used for training.
• The prediction error (and standard deviation) as a function of the number of data points used for training.
5. Write a report describing the implemented algorithms and discussing the results and the learned lessons.

In [1]:
# if f(x) >= 0 then y = face
# else y = not face

#f(x) = w1*phi(x) + w2*phi2(x) + ..
# phi(x) = number of black pixels (example)

# intialize weights randomonly at first

# training condition is while less than 5 minutes or
# the weights aren't changing from image to image

#  update weights in training
 # if f(x) >=0 and y =face
 # or  f(x) <= 0 and y = not face
 # then do nothing

 # else if f(x) >= 0 and y = not face
 # then w1 = w1 -phi1 (x) 
 #      w2 = w2 - phi2(x)

# else if f(x) < 0 and y = face
 # then w1 = w1 + phi1 (x) 
 #      w2 = w2 + phi2(x)


#f = w1 * phi1(x) + w2 * phi2(x)



In [2]:
# Determine features - will need to obtain from image
# First step is get data from facedatatrain to use in this notebook
# Each face image is 70x60 (row x column) and each face begins and ends with an empty line (not including the 70x60).
# Then make a feature matrix of size 70*60, where each pixel is a feature

In [3]:
! pip install numpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\nafri\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import sys
sys.path.insert(1, 'C:/Users/nafri/College/Senior Year/Spring Semester/Intro to AI/AI-FinalProject/Final Project/Berkeley_Example')
sys.path.insert(1, 'C:/Users/jzlyn/OneDrive/Desktop/CS440/AI-FinalProject/Final Project/Berkeley_Example')

In [29]:
from Berkeley_Example import samples
from Berkeley_Example.perceptron import PerceptronClassifier
import numpy as np
import math

In [41]:
def sigmoid(x):
    try:
        sig =1/(1 + math.exp(-x))
    except:
        sig = 0   
    return sig

In [45]:
# Training data
# Should put into function and clean up code


# Parses images and divides image into features
NUM_IMAGES = 451
face_train_images = samples.loadDataFile('data/facedata/facedatatrain', NUM_IMAGES, 60, 70)
face_train_labels = samples.loadLabelsFile('data/facedata/facedatatrainlabels', NUM_IMAGES)
# face_valid_images = samples.loadDataFile('data/facedata/facedatavalidation', 10, 60, 70)
# face_valid_labels = samples.loadLabelsFile('data/facedata/facedatavalidationlabels', 10)

image_lst = []
"""
switch = True
for image in face_train_images:
    while switch:
        print(image.getPixels())
        switch = False
"""
# Create list of 2d lists that has feature vals for images
for image in face_train_images:
    matrix = image.getPixels()
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            matrix[i][j] = samples.IntegerConversionFunction(matrix[i][j])
    image_lst.append(matrix)


#print(image_lst[0])

# Randomly intialize weights and bias

# weights = np.random.randint(-400,400,size=(70,60))
# bias = np.random.randint(-200,200)

weights = np.random.uniform(-1, 1, size=(70, 60))
bias = np.random.uniform(-1, 1)
#print(weights)
#print(bias)

#f = np.zeros(NUM_IMAGES).astype(int)
f = np.zeros(NUM_IMAGES).astype(float)

#print(f)

'''with float and uniform(-1,1):
iteration = 1
Model Accuracy = 52.33%
iteration = 2
Model Accuracy = 51.33%
iteration = 3
Model Accuracy = 52.67%
iteration = 4
Model Accuracy = 54.33%
iteration = 5
Model Accuracy = 51.33%
...
iteration = 14
Model Accuracy = 50.67%
iteration = 15
Model Accuracy = 50.67%
'''
'''with int
iteration = 1
Model Accuracy = 48.33%
iteration = 2
Model Accuracy = 49.67%
iteration = 3
Model Accuracy = 48.33%
iteration = 4
Model Accuracy = 49.00%
iteration = 5
Model Accuracy = 49.00%
iteration = 6
Model Accuracy = 49.33%
iteration = 7
Model Accuracy = 49.00%
...
iteration = 14
Model Accuracy = 49.00%
iteration = 15
Model Accuracy = 49.00%'''

def update_weights(image,weights,over_estimate):
    if over_estimate:
        #bias -= 1
        for i in range(len(weights)):
            for j in range(len(weights[0])):
                #print(f'Before: {weights[i][j]} Feature: {image[i][j]}')
                weights[i][j] -= image[i][j]
                #print(f'After: {weights[i][j]}')
    else:
        #bias += 1
        for i in range(len(weights)):
            for j in range(len(weights[0])):
                #print(f'Before: {weights[i][j]} Feature: {image[i][j]}')
                weights[i][j] += image[i][j]
                #print(f'After: {weights[i][j]}')

#i = 0
#while i < len(image_lst):

# Need to make outer loop that iterate until time limit is hit or weights have changed since last iteration (weights have converged)
iteration = 0

# Need to make it show we train algo on 10%, 20%, ..., 100% of training data and store weights and biases to use on validation data

while iteration < 10: 
    iteration += 1
    print(f'iteration = {iteration}')

    for i in range(len(image_lst)):
        #bias = np.random.uniform(-1, 1)
        #print(f'bias = {bias}')
        f[i] = bias
        #print(f'f(x{i}) = {f[i]}')
        #print(f'f{f}')
        image = image_lst[i]
        #print(image)
        for r in range(len(image)):
            #print(len(image))
            for c in range(len(image[0])):
                #print(len(image[0]))
                #print(f'weight {weights[r][c]} image {image[r][c]}')
                f[i] += weights[r][c] * image[r][c]
        #print(f[i])
        f[i] = sigmoid(f[i])
        #print(f[i])
        #print(f'f(x{i}) = {f[i]}, Label = {face_train_labels[i]}')

        #if (f >= 0 and face_train_labels[i] == 1) or (f < 0 and face_train_labels[i] == 0):
            #continue
        if round(f[i]) == 1 and face_train_labels[i] == 0:
            #print('Overestimate')
            #print(f'f={f[i]},actual={face_train_labels[i]}')
            #print(f'Before Bias (-) = {bias}')
            update_weights(image,weights,True)
            bias -= 1
            #print(f'After Bias (-) = {bias}')


        elif round(f[i]) ==  0 and face_train_labels[i] == 1:
            #print('Underestimate')
            #print(f'f={f[i]},actual={face_train_labels[i]}')
            #print(f'Before Bias (+) = {bias}')
            update_weights(image,weights,False)
            bias += 1
            #print(f'After Bias (+) = {bias}')
      
    '''
    num_correct = 0
    for i in range(len(f)):
        print(f[i],face_train_labels[i])
        if (f[i] >= 0 and face_train_labels[i] == 1) or (f[i] < 0 and face_train_labels[i] == 0):
            num_correct += 1
    #print(num_correct)

    result = '{:.2%}'.format(num_correct / len(face_train_labels))
    
    print(f'Model Accuracy = {result}')
    '''


    #for score in f:
        #print(score)


# Need to create data structure to hold f so I can compare f of each image to proper answer (train_labels) to measure accuracy

#face_perceptron = PerceptronClassifier(legal_labels,10)
#face_perceptron.train(face_train_images,face_train_labels,face_valid_images,face_valid_labels)

iteration = 1
iteration = 2
iteration = 3
iteration = 4
iteration = 5
iteration = 6
iteration = 7
iteration = 8
iteration = 9
iteration = 10


In [46]:
# Validation
# Should make these cells into functions and clean up the code
NUM_IMAGES =301
face_valid_images = samples.loadDataFile('data/facedata/facedatavalidation', NUM_IMAGES, 60, 70)
face_valid_labels = samples.loadLabelsFile('data/facedata/facedatavalidationlabels', NUM_IMAGES)


image_lst = []
"""
switch = True
for image in face_train_images:
    while switch:
        print(image.getPixels())
        switch = False
"""
# Create list of 2d lists that has feature vals for images
for image in face_valid_images:
    matrix = image.getPixels()
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            matrix[i][j] = samples.IntegerConversionFunction(matrix[i][j])
    image_lst.append(matrix)

f = np.zeros(NUM_IMAGES).astype(float)

#iteration = 0

#while iteration < 10:
#iteration += 1
#print(f'iteration = {iteration}')

for i in range(len(image_lst)):
    #bias = np.random.uniform(-1, 1)
    #print(f'bias = {bias}')
    f[i] = bias
    #print(f'f(x{i}) = {f[i]}')
    #print(f'f{f}')
    image = image_lst[i]
    #print(image)
    for r in range(len(image)):
        #print(len(image))
        for c in range(len(image[0])):
            #print(len(image[0]))
            #print(f'weight {weights[r][c]} image {image[r][c]}')
            f[i] += weights[r][c] * image[r][c]

num_correct = 0
for i in range(len(f)):
    print(f[i],face_valid_labels[i])
    if (f[i] >= 0 and face_valid_labels[i] == 1) or (f[i] < 0 and face_valid_labels[i] == 0):
        num_correct += 1
#print(num_correct)

result = '{:.2%}'.format(num_correct / len(face_valid_labels))

print(f'Model Accuracy = {result}')

34.2830812681872 1
-1470.5348512742587 0
-365.89556076804547 0
1304.8002140827573 1
1522.8057350927706 1
851.8291947916106 1
-410.5514690223209 0
106.50814382950614 0
-425.071712363035 0
-588.8294053743267 0
430.07851568323474 0
-245.6567276501293 0
928.9727902961257 1
854.70354738718 1
-825.7427094159957 0
-1291.675700847303 0
48.484318103628894 1
516.3208513043127 1
680.3428036858478 1
917.5662548230632 1
-619.6599326898365 0
359.2339738017405 1
29.020350643218094 0
284.54643279034826 1
131.5791290772094 1
-1709.4795169639513 0
-65.73542181563174 0
-198.332624528092 0
19.68652839304937 1
529.8468383995566 1
429.60882532799843 1
706.7943148790633 1
68.94690243516366 1
-1315.0454164652695 0
-443.3967171243533 0
-1035.6729935819735 0
-929.849633891265 0
926.4635271172671 1
718.8222021861002 1
-798.4584551352293 0
-689.4427175936177 0
-918.3014529870529 0
-289.3177380128518 0
-734.3264858001505 0
-864.6008869058305 0
75.80814030816003 0
306.19835182579465 1
-394.54049283589376 1
3.897131

In [47]:
import numpy as np
weights = np.random.rand(1, 4200)
phitures = 0